<a href="https://colab.research.google.com/github/Ylieo816/US-Stock-Investion-Calculator/blob/main/ETF_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Basic Info**

Make a copy for the following sheet on you google drive and edit that copyed file (DO NOT EDIT FILE NAME):

https://docs.google.com/spreadsheets/d/1CJCSn11gKZ2P7JkRzmSoizO--zyscO-s/edit?usp=sharing&ouid=110680874934690356046&rtpof=true&sd=true

## **Results**

Please view the same file for the results (File in your Google Drive, Copyed File)

Other Sheets Explaination (After Excution the Program):

### auto re-balance output:
- Offers a detailed breakdown for each asset, showcasing its current price and the number of shares to buy or sell post-rebalancing.

### Extra Detail:
- Provides insights into individual assets and presents a comprehensive summary for each asset as well as an aggregate summary for all assets combined.



## **CODE**

In [ ]:
!pip install pandas numpy
from google.colab import drive
drive.mount('/content/drive')
!pip install yfinance
!pip install openpyxl
!pip install xlsxwriter
!pip install numpy_financial
!pip install yagmail


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from IPython.display import display, Javascript
from openpyxl import load_workbook
from openpyxl.styles import Alignment

def alert(message):
    display(Javascript(f'alert("{message}")'))

def center_content_in_sheet(workbook, sheet_name):
    ws = workbook[sheet_name]
    for row in ws.iter_rows():
        for cell in row:
            cell.alignment = Alignment(horizontal='center', vertical='center')

errors = []  # List to collect errors

# Load the data from the Excel file
df = pd.read_excel('/content/drive/My Drive/Previous_ETF_Record.xlsx', sheet_name='auto re-balance input')

# Load the workbook
wb = load_workbook('/content/drive/My Drive/Previous_ETF_Record.xlsx')

# Check if the sheet exists
if 'ETF Prediction' in wb.sheetnames:
    # If it exists, remove it
    wb.remove(wb['ETF Prediction'])

# Check if the sheet exists
if 'Shares To Buy' in wb.sheetnames:
    # If it exists, remove it
    wb.remove(wb['Shares To Buy'])

# Check if the sheet exists
if 'Performance' in wb.sheetnames:
    # If it exists, remove it
    wb.remove(wb['Performance'])

# Check if the sheet exists
if 'Error' in wb.sheetnames:
    # If it exists, remove it
    wb.remove(wb['Error'])

# Check if the sheet exists
if 'auto re-balance output' in wb.sheetnames:
    # If it exists, remove it
    wb.remove(wb['auto re-balance output'])

# Check if the sheet exists
if 'Extra Detail' in wb.sheetnames:
    # If it exists, remove it
    wb.remove(wb['Extra Detail'])

# Save the workbook
wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')


# Save the workbook
wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')
# Check that the required columns exist
required_columns = ['Symbol', 'Weight Allocation', 'Pre-txn Quantity (交易前數量)', 'Investment Budget (USD) (賣出-/投資+)']
for column in required_columns:
    if column not in df.columns:
        errors.append(f"Missing required column: {column}")

# Set the variables based on the data in the DataFrame
symbols_list = df['Symbol'].tolist()
distribution = df['Weight Allocation'].tolist()
current_ETF_shares = df['Pre-txn Quantity (交易前數量)'].tolist()
# last_investment_date = '2023-04-04 00:00:00'
original_amount_available = df['Investment Budget (USD) (賣出-/投資+)'].iloc[0]

# Convert the lists to contain float values
distribution = [float(x) for x in distribution]
current_ETF_shares = [0 if np.isnan(x) else float(x) for x in current_ETF_shares]

# Convert original_amount_available to float
original_amount_available = float(original_amount_available)

print(distribution)

# Check that the distributions add up to 1 (allowing for a small error)
if not np.isclose(sum(distribution), 1, atol=0.001):
    errors.append("The total weights do not add up to 1 or 100%.")

# Check that there are no negative values
if any(x < 0 for x in current_ETF_shares):
    errors.append("The current ETF shares contain negative values.")

# Define the investment dates 2023, investing on the first business day of each quarter
dates_list = pd.date_range(start = pd.to_datetime('today').strftime('%Y-%m-%d'), end = pd.to_datetime('today').strftime('%Y-%m-%d')).strftime('%Y-%m-%d').tolist()
dates_list.insert(0, "2023-10-01")
dates_list = [pd.to_datetime(date).date() for date in dates_list]

# Initialize an empty DataFrame with dates as the index
price_data = pd.DataFrame(index=pd.to_datetime(dates_list))
url = "https://www.x-rates.com/calculator/?from=TWD&to=USD&amount=1"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
rate = soup.find_all('span', {'class': 'ccOutputTrail'})[0].previous_sibling
TWD_USD_rate = float(rate)

# Fetch the historical price data for each symbol and add it to the DataFrame
for symbol in symbols_list:
    # Fetch the historical price data
    try:
        tickerData = yf.Ticker(symbol)
        tickerDf = tickerData.history(period='1d')

        # Convert the index of tickerDf to a timezone-naive datetime index
        tickerDf.index = tickerDf.index.tz_convert(None)

        if symbol == '0050.TW':
            price_data[symbol] = tickerDf['Close'].reindex(price_data.index, method='ffill') * TWD_USD_rate
            # Update the last row with the current price
            price_data.loc[price_data.index[-1], symbol] = tickerDf['Close'].iloc[-1] * TWD_USD_rate
        else:
            price_data[symbol] = tickerDf['Close'].reindex(price_data.index, method='ffill')
            # Update the last row with the current price
            price_data.loc[price_data.index[-1], symbol] = tickerDf['Close'].iloc[-1]
    except Exception as e:
        errors.append(f"An error occurred while fetching data for {symbol}. Please double check if the {symbol} exists.")

# price_data.index = pd.to_datetime(price_data.index)
price_data.index = price_data.index.date
price_data.index.name = 'Date'

# Remove the row for "2023-01-01" from price_data if it exists
if pd.Timestamp("2023-01-01").date() in price_data.index:
    price_data = price_data.drop(pd.Timestamp("2023-01-01"))

hi = dates_list.pop(0)
print(current_ETF_shares)


[0.03, 0.1, 0.01, 0.01, 0.05, 0.01, 0.0829, 0.01, 0.03, 0.02, 0.01, 0.03, 0.3, 0.2971, 0.01, 0.0, 0.0, 0.0, 0.0]
[3.01, 6.01, 5.01, 3.01, 0.0, 6.01, 9.02, 0, 0, 0, 0, 0, 0, 0, 0, 3.01, 0.0, 1.0, 5.02]


In [ ]:
from scipy.optimize import brentq

def calculate_difference(amount_available):
  curr_value = 0
  # Initialize a dictionary to hold the data for this date
  date_data = {'Date': pd.to_datetime('today').strftime('%Y-%m-%d')}
  predict_data = {'Date': []}
  predict_data['Symbol'] = []
  predict_data['Desired Value'] = []
  predict_data['Price'] = []
  predict_data['Shares Before'] = []
  predict_data['Total Value Before'] = []
  predict_data['Amount to Invest'] = []
  predict_data['Shares to Buy'] = []
  predict_data['Shares After'] = []
  date_tem = date_data['Date']

  for z in range(len(symbols_list)):
      # Get the current symbol and distribution
      symbol = symbols_list[z]

      # Get the current price and number of shares owned before rebalancing
      price = price_data.loc[date, symbol]
      if i == 0:
          shares_before = 0
      else:
          shares_before = current_ETF_shares[z]

      # Calculate the total value before rebalancing
      if symbol == '0050.TW':
          curr_total_value = price * shares_before * 1
      else:
          curr_total_value = price * shares_before
      # Sum up
      curr_value +=  curr_total_value

  total_amount = amount_available + curr_value

  # Iterate over each ETF or Bond
  for j in range(len(symbols_list)):
      # Get the current symbol and distribution
      symbol = symbols_list[j]
      dist = distribution[j]
      date_tem = pd.to_datetime(date)

      # Calculate the desired value
      desired_value = total_amount * dist

      # Get the current price and number of shares owned before rebalancing
      price = price_data.loc[date, symbol]
      if i == 0:
          shares_before = 0
      else:
          shares_before = float(current_ETF_shares[j])

      # Calculate the total value before rebalancing
      if symbol == '0050.TW':
          total_value_before = price * shares_before * 1
      else:
          total_value_before = price * shares_before

      # Calculate the amount to invest (or redeem)
      amount_to_invest = desired_value - total_value_before


      # Calculate shares to buy as before
      if symbol == '0050.TW':
          shares_to_buy = round(amount_to_invest / (price * 1))
      else:
          shares_to_buy = round(amount_to_invest / price)

      # Ensure that we're not trying to sell more shares than we own
      if shares_to_buy < 0 and -shares_to_buy > shares_before:
          # print('here')
          shares_to_buy = -int(shares_before)


      # Calculate the number of shares owned after rebalancing
      shares_after = float(shares_before + shares_to_buy)

      # Add the calculations to the dictionary
      date_data[f'{symbol} Desired Value'] = desired_value
      date_data[f'{symbol} Price'] = price
      date_data[f'{symbol} Shares Before'] = shares_before
      date_data[f'{symbol} Total Value Before'] = total_value_before
      date_data[f'{symbol} Amount to Invest'] = amount_to_invest
      date_data[f'{symbol} Shares to Buy'] = shares_to_buy
      date_data[f'{symbol} Shares After'] = shares_after

      predict_data['Date'].append(pd.to_datetime('today').strftime('%Y-%m-%d'))
      predict_data['Symbol'].append(symbol)
      predict_data['Desired Value'].append(desired_value)
      predict_data['Price'].append(price)
      predict_data['Shares Before'].append(shares_before)
      predict_data['Total Value Before'].append(total_value_before)
      predict_data['Amount to Invest'].append(amount_to_invest)
      predict_data['Shares to Buy'].append(shares_to_buy)
      predict_data['Shares After'].append(shares_after)

  # Calculate actual_invest_amount
  actual_invest_amount = 0
  for j in range(len(symbols_list)):
      # Get the current symbol
      symbol = symbols_list[j]

      # Get the current price and number of shares to buy
      price = price_data.loc[date, symbol]
      shares_to_buy = date_data[f'{symbol} Shares to Buy']  # Use date instead of i

      # Calculate the value
      value = 0
      if symbol == '0050.TW':
          value = price * shares_to_buy * 1
      else:
         value = shares_to_buy * price
      actual_invest_amount += value
  calculate_difference.date_data = date_data
  calculate_difference.predict_data = predict_data
  calculate_difference.actual_invest_amount= actual_invest_amount
  calculate_difference.difference = actual_invest_amount - original_amount_available
  return actual_invest_amount - original_amount_available


#####################
##Rebalancing Start##
#####################

amount_available = original_amount_available

date = dates_list[-1]

if (len(errors) == 0):

    # Calculate the total value of the portfolio
    total_portfolio_value = sum(price_data.loc[date, symbol] * shares for symbol, shares in zip(symbols_list, current_ETF_shares))

    # Check that the user is not trying to sell more than they own
    if original_amount_available < -total_portfolio_value:
        errors.append(f"You are trying to sell more than you currently own.")

# Initialize the best result
best_result = None
best_difference = float('inf')
best_predict_data = None
best_date_data = None
i = 1
date_data = None
best_date_data = None

# Set a tolerance for the difference
tolerance = 1e-6

if (len(errors) == 0):

    # Run the initial amount
    amount_available = original_amount_available
    difference = calculate_difference(amount_available)

    # If the difference is negative, increment the amount
    if difference < 0:
        # Initialize a counter for the number of consecutive decreases
        decrease_counter = 0

        while True:
            # Increment the amount
            amount_available += 1

            # Calculate the new difference
            new_difference = calculate_difference(amount_available)

            # If this result is better than the current best result, update the best result
            if abs(new_difference) < best_difference:
                best_difference = abs(new_difference)
                best_result = calculate_difference.actual_invest_amount
                best_predict_data = calculate_difference.predict_data
                best_date_data = calculate_difference.date_data

            # If the new difference is less than the old difference, increment the counter
            if new_difference > 0 :
                decrease_counter += 1
            else:
                # If the new difference is not less than the old difference, reset the counter
                decrease_counter = 0

            # If the new difference is close to zero or the counter reaches a certain threshold, stop the loop
            if abs(new_difference) < tolerance or decrease_counter >= 5:  # Change 5 to the number of consecutive decreases you want
                break

            # Update the difference
            difference = new_difference

    # If the difference is positive, decrement the amount
    elif difference > 0:
        # Initialize a counter for the number of consecutive decreases
        decrease_counter = 0

        while True:
            # Decrement the amount
            amount_available -= 1

            # Calculate the new difference
            new_difference = calculate_difference(amount_available)
            # If this result is better than the current best result, update the best result
            if abs(new_difference) < best_difference:
                best_difference = abs(new_difference)
                best_result = calculate_difference.actual_invest_amount
                best_predict_data = calculate_difference.predict_data
                best_date_data = calculate_difference.date_data

            # If the new difference is less than the old difference, increment the counter
            if new_difference < 0:
                decrease_counter += 1
            else:
                # If the new difference is not less than the old difference, reset the counter
                decrease_counter = 0

            # If the new difference is close to zero or the counter reaches a certain threshold, stop the loop
            if abs(new_difference) < tolerance or decrease_counter >= 5:  # Change 5 to the number of consecutive decreases you want
                break

            # Update the difference
            difference = new_difference

    # Access updated date_data
    iter_date_data = best_predict_data
    iter_best_date_data = best_date_data
    iter_best_difference = best_difference

    # Print the best result
    print(f"The best result with iteration is: {best_result}")


    from scipy.optimize import root_scalar

    # Use the difference from your iterative method as the tolerance
    tolerance = iter_best_difference

    # Check the signs of the function at the endpoints of the bracket
    fa = calculate_difference(0)
    fb = calculate_difference(original_amount_available * 1.5)

    if np.sign(fa) == np.sign(fb):
        date_data = iter_date_data
        best_date_data = iter_best_date_data

    else:
        _ = root_scalar(calculate_difference, method='brentq', bracket=[0, original_amount_available * 1.5], xtol=tolerance)
        brentq_date_data = calculate_difference.predict_data
        brentq_best_date_data = calculate_difference.date_data
        brentq_best_difference = calculate_difference.difference

        # Print the best result
        print(f"The best result with brentq is: {calculate_difference.actual_invest_amount}")

        print(iter_best_difference , '  ', abs(brentq_best_difference))

        if (iter_best_difference <= abs(brentq_best_difference)):
            date_data = iter_date_data
            best_date_data = iter_best_date_data
        else:
            date_data = brentq_date_data
            best_date_data = brentq_best_date_data


print(date_data)
print(best_date_data)


The best result with iteration is: 2041.9038734817505
The best result with brentq is: 1994.5038719558715
3.623873481750479    43.77612804412843
{'Date': ['2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10', '2023-09-10'], 'Symbol': ['VEU', 'VTI', 'VB', 'VNQ', '0050.TW', 'VWO', 'VTIP', 'BND', 'VCIT', 'HYG', 'VGIT', 'VIG', 'VGT', 'XLV', 'ESGU', 'VGK', 'VSS', 'BSV', 'VCSH'], 'Desired Value': [181.77684603767392, 605.9228201255798, 60.59228201255798, 60.59228201255798, 302.9614100627899, 60.59228201255798, 502.31001788410566, 60.59228201255798, 181.77684603767392, 121.18456402511596, 60.59228201255798, 181.77684603767392, 1817.7684603767393, 1800.1966985930974, 60.59228201255798, 0.0, 0.0, 0.0, 0.0], 'Price': [53.22999954223633, 221.4199981689453, 196.25999450683594, 81.58000183105469, 3.866973

In [ ]:
if(len(errors) == 0):

    # Add TWD_USD_rate to the date_data dictionary
    date_data['TWD_USD_rate'] = TWD_USD_rate

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(date_data)

    # Switch the sign of 'Amount to Invest'
    df['Amount to Invest'] = df['Amount to Invest'] * -1

    # Load the workbook
    wb = load_workbook('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    # Check if the sheet exists
    if 'ETF Prediction' in wb.sheetnames:
        # If it exists, remove it
        wb.remove(wb['ETF Prediction'])

    # Save the workbook
    wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    # For the ETF Prediction data
    with pd.ExcelWriter('/content/drive/My Drive/Previous_ETF_Record.xlsx', engine='openpyxl', mode='a') as writer:
        df.to_excel(writer, sheet_name='ETF Prediction', index=False)

    # Load the workbook
    wb = load_workbook('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    # Access the sheet
    ws = wb['ETF Prediction']

    # Iterate over the columns
    for column in ws.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column[0].column_letter].width = adjusted_width

    # Save the workbook
    wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    date_data_RE = best_date_data # calculate_difference.date_data
    # Create a new DataFrame for the "Shares To Buy" data
    shares_to_buy_data = {'Symbol': symbols_list}
    shares_to_buy_data['Shares To Buy'] = [date_data_RE[f'{symbol} Shares to Buy'] for symbol in symbols_list]

    df_shares_to_buy = pd.DataFrame(shares_to_buy_data)

    # Load the workbook
    wb = load_workbook('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    # Check if the sheet exists
    if 'Shares To Buy' in wb.sheetnames:
        # If it exists, remove it
        wb.remove(wb['Shares To Buy'])

    # Save the workbook
    wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    # Now you can write to the sheet
    with pd.ExcelWriter('/content/drive/My Drive/Previous_ETF_Record.xlsx', engine='openpyxl', mode='a') as writer:
        df_shares_to_buy.to_excel(writer, sheet_name='Shares To Buy', index=False)

    # Load the workbook
    wb = load_workbook('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    # Access the sheet
    ws = wb['Shares To Buy']

    # Iterate over the columns
    for column in ws.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column[0].column_letter].width = adjusted_width

    # Save the workbook
    wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')

In [ ]:
if(len(errors) == 0):

    # Initialize an empty DataFrame to hold the value calculations
    performance_data_1 = pd.DataFrame()
    total_value_before_invest = []

    # Iterate over each date

    date = dates_list[-1]

    # Initialize a dictionary to hold the calculations for this date
    date_data = {'Date': date}
    tem_total_before = 0
    # Iterate over each ETF or Bond
    for j in range(len(symbols_list)):
        # Get the current symbol
        symbol = symbols_list[j]

        # Get the current price and number of shares to buy
        price = price_data.loc[date, symbol]
        shares_to_buy = date_data_RE[f'{symbol} Shares to Buy']  # Use date instead of i
        shares_before = date_data_RE[f'{symbol} Shares Before']  # Use date instead of i

        # Calculate the value
        if symbol == '0050.TW':
            value = shares_to_buy * price * -1
            tem_total_before += (shares_before * price * 1)
        else:
            value = shares_to_buy * price * -1
            tem_total_before += (shares_before * price)

        # Add the value to the dictionary
        date_data[f'{symbol} Performance'] = value
    total_value_before_invest.append(tem_total_before)
    # Add the dictionary to the DataFrame
    performance_data_1 = pd.concat([performance_data_1, pd.DataFrame(date_data, index=[0])], ignore_index=True)

    # Set the date as the index of the DataFrame
    performance_data_1.set_index('Date', inplace=True)

    invest_value = []

    # Get the current date
    date = dates_list[-1]

    # Calculate the total value of all ETFs and bonds for this date
    invest_value.append(performance_data_1.loc[date].sum())

    performance_data_1.insert(len(performance_data_1.columns), 'Total Value', invest_value)
    performance_data_1.insert(0, 'Amount Available', original_amount_available)
    performance_data_1.insert(1, 'Total Value Before Invest', total_value_before_invest)

    difference_list = []
    within_range_list = []

    # Calculate the difference and check if it's within the range
    for i in range(len(performance_data_1)):
        invest_value = performance_data_1.iloc[i, performance_data_1.columns.get_loc('Total Value')] * -1
        amount_available = performance_data_1.iloc[i, performance_data_1.columns.get_loc('Amount Available')]
        # difference = (invest_value - amount_available) / amount_available
        difference = (invest_value - amount_available)
        difference_list.append(difference)
        within_range_list.append('Yes' if 0.05 >= difference >= -0.05 else 'No')

    performance_data_1.insert(len(performance_data_1.columns), 'Difference', difference_list)
    performance_data_1.insert(len(performance_data_1.columns), 'Within Range', within_range_list)

    # Print the DataFrame to check
    # print(performance_data_1)

    # Load the workbook
    wb = load_workbook('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    # Check if the sheet exists
    if 'Performance' in wb.sheetnames:
        # If it exists, remove it
        wb.remove(wb['Performance'])

    # Save the workbook
    wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    # Now you can write to the sheet
    with pd.ExcelWriter('/content/drive/My Drive/Previous_ETF_Record.xlsx', engine='openpyxl', mode='a') as writer:
        performance_data_1.to_excel(writer, sheet_name='Performance', index=False)

    # Load the workbook
    wb = load_workbook('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    # Access the sheet
    ws = wb['Performance']

    from openpyxl.styles import PatternFill

    # Define a red fill
    red_fill = PatternFill(start_color="FFFF0000", end_color="FFFF0000", fill_type="solid")

    # Iterate over the rows
    for row in ws.iter_rows(min_row=2, min_col=ws.max_column - 1, max_col=ws.max_column):
        for cell in row:
            # If the cell value is 'No', fill it with red
            if cell.value == 'No':
                cell.fill = red_fill

    # Iterate over the columns
    for column in ws.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column[0].column_letter].width = adjusted_width

    # Save the workbook
    wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')


    # print(performance_data_1)


    # print(df)

In [ ]:
if(len(errors) == 0):

    # Convert the dictionary to a DataFrame for ETF Prediction
    # df_etf_prediction = pd.DataFrame(date_data)

    # Create a new DataFrame for the "Shares To Buy" data
    shares_to_buy_data = {'Symbol': symbols_list}
    shares_to_buy_data['Shares To Buy'] = [date_data_RE[f'{symbol} Shares to Buy'] for symbol in symbols_list]
    df_shares_to_buy = pd.DataFrame(shares_to_buy_data)

    # Combine ETF Prediction and Performance data
    combined_df = pd.concat([df, performance_data_1], axis=0, ignore_index=True)

    # Load the workbook
    wb = load_workbook('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    # Check if the combined sheet exists
    if 'Extra Detail' in wb.sheetnames:
        # If it exists, remove it
        wb.remove(wb['Extra Detail'])

    # Save the workbook
    wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    # Now you can write the combined data to the sheet
    with pd.ExcelWriter('/content/drive/My Drive/Previous_ETF_Record.xlsx', engine='openpyxl', mode='a') as writer:
        combined_df.to_excel(writer, sheet_name='Extra Detail', index=False)

    # Load the workbook
    wb = load_workbook('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    # Access the sheet
    ws = wb['Extra Detail']

    # Iterate over the columns to adjust width
    for column in ws.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column[0].column_letter].width = adjusted_width

    # Save the workbook
    wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    wb = load_workbook('/content/drive/My Drive/Previous_ETF_Record.xlsx')

    # Remove the original sheets
    sheets_to_remove = ['ETF Prediction', 'Performance']
    for sheet in sheets_to_remove:
        if sheet in wb.sheetnames:
            wb.remove(wb[sheet])

    # Save the workbook
    wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')


In [ ]:
# Check if best_date_data is None
if best_date_data is None:
    shares_to_buy = [None] * len(symbols_list)
    pre_txn_quantity = [None] * len(symbols_list)
    unit_prices = [None] * len(symbols_list)
    total_investment = None
else:
    shares_to_buy = [best_date_data[f'{symbol} Shares to Buy'] for symbol in symbols_list]
    pre_txn_quantity = [best_date_data[f'{symbol} Shares Before'] for symbol in symbols_list]
    unit_prices = [price_data.loc[dates_list[-1], symbol] for symbol in symbols_list]
    total_investment = sum([best_date_data[f'{symbol} Shares to Buy'] * -1 * price for symbol, price in zip(symbols_list, unit_prices)])

# Create a new DataFrame for the result presentation
result_data = {
    'Index': range(1, len(symbols_list) + 1),
    'Symbol (input)': symbols_list,
    'Weight Allocation': distribution,
    'Pre-txn Quantity (交易前數量)': current_ETF_shares,
    'Date & Time': [pd.to_datetime('today').strftime('%Y-%m-%d %H:%M:%S')] * len(symbols_list),
    'Symbol (output)': symbols_list,
    'Unit Price': unit_prices,
    'Shares to 賣出-/買入+': shares_to_buy
}

df_result = pd.DataFrame(result_data)

# Add the 'Investment Budget (USD) (賣出-/投資+)' column to the DataFrame
# Only the first row contains the total investment, the rest are NaN
df_result['Est. Investment Amt. (USD) (所得+ /付出-)'] = [total_investment] + [float('nan')] * (len(df_result) - 1)

# Pad the errors list with empty strings to match the length of symbols_list
if(len(errors) == 0):
  # Add the 'Status' column.
  df_result['Status'] = ['complete'] * (len(symbols_list) - len(errors))
else:
  errors += [''] * (len(symbols_list) - len(errors))
  # Add the 'Status' column.
  df_result['Status'] = errors

# Load the workbook
wb = load_workbook('/content/drive/My Drive/Previous_ETF_Record.xlsx')

# Check if the sheet "auto re-balance output" exists, if yes, remove it
if 'auto re-balance output' in wb.sheetnames:
    wb.remove(wb['auto re-balance output'])

# Save the workbook
wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')

# Write the auto re-balance output data to the sheet
with pd.ExcelWriter('/content/drive/My Drive/Previous_ETF_Record.xlsx', engine='openpyxl', mode='a') as writer:
    df_result.to_excel(writer, sheet_name='auto re-balance output', index=False)

# Load the workbook
wb = load_workbook('/content/drive/My Drive/Previous_ETF_Record.xlsx')

# Access the sheet
ws = wb['auto re-balance output']

# Iterate over the columns to adjust column width
for column in ws.columns:
    max_length = 0
    column = [cell for cell in column]
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column[0].column_letter].width = adjusted_width

# Save the workbook
wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')


In [ ]:
wb = load_workbook('/content/drive/My Drive/Previous_ETF_Record.xlsx')

if errors:
    # If there's an error, only keep 'auto-rebalance input' and 'auto-rebalance output' sheets
    sheets_to_keep = ['auto re-balance input', 'auto re-balance output']
    for sheet in wb.sheetnames:
        if sheet not in sheets_to_keep:
            wb.remove(wb[sheet])
    center_content_in_sheet(wb, 'auto re-balance output')


else:
    # If there's no error, execute the original code
    sheets_to_remove = ['ETF Prediction', 'Performance','Shares To Buy']
    for sheet in sheets_to_remove:
        if sheet in wb.sheetnames:
            wb.remove(wb[sheet])

    # Center the content of the sheet
    center_content_in_sheet(wb, 'auto re-balance output')
    center_content_in_sheet(wb, 'Extra Detail')

    # Check if the sheets exist
    if 'auto re-balance input' in wb.sheetnames and 'auto re-balance output' in wb.sheetnames and 'Extra Detail' in wb.sheetnames:
        # Reorder the sheets
        wb._sheets = [wb['auto re-balance input'], wb['auto re-balance output'], wb['Extra Detail']] + [s for s in wb._sheets if s.title not in ['auto re-balance input', 'auto re-balance output', 'Extra Detail']]

# Save the workbook
wb.save('/content/drive/My Drive/Previous_ETF_Record.xlsx')
